In [2]:
import os
import cv2
import numpy as np
from datetime import timedelta
from matplotlib import pyplot as plt
import matplotlib

In [4]:
data_loc = "compressor-data/"
files = os.listdir(data_loc)

files_len = len(files)
print(files_len)

MAX_FRAMES_PER_SECOND = 10

803


In [5]:
def format_timedelta(td):
    result = str(td)
    try:
        result, ms = result.split(".")
    except ValueError:
        return (result + ".00").replace(":", "-")
    ms = int(ms)
    ms = round(ms / 1e4)
    return f"{result}.{ms:02}".replace(":","-")

In [6]:
def get_saving_frames_durations(cap, saving_fps):
    s = []
    clip_duration = cap.get(cv2.CAP_PROP_FRAME_COUNT) / cap.get(cv2.CAP_PROP_FPS)

    for i in np.arange(0, clip_duration, 1/saving_fps):
        s.append(i)
    
    return s

In [7]:
def video_analyze(file, override=False):
    video = cv2.VideoCapture(file)
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    print("Total frames = " + str(total_frames))

    fps = video.get(cv2.CAP_PROP_FPS)
    print("FPS = " + str(fps))

In [8]:
def video_into_frames(video_file):
    folder_name = 'images'
    
    cap = cv2.VideoCapture(video_file)
    fps = cap.get(cv2.CAP_PROP_FPS)
    saving_frames_per_second = min(fps, MAX_FRAMES_PER_SECOND)
    saving_frames_duration = get_saving_frames_durations(cap, saving_frames_per_second)

    count = 0
    while True:
        is_read, frame = cap.read()
        if not is_read:
            break
            
        frame_duration = count/fps
        try:
            closest_duration = saving_frames_duration[0]
        except IndexError:
            break

        if frame_duration >= closest_duration:
            frame_duration_formatted = format_timedelta(timedelta(seconds=frame_duration))
            file_name = str(video_file).split('/')[1]
            cv2.imwrite(os.path.join(folder_name, f"{file_name}-{frame_duration_formatted}.jpg"), frame)
            try:
                saving_frames_duration.pop(0)
            except IndexError:
                pass
        
        count += 1

In [7]:
video_list = []
for file in os.listdir('compressor-data'):
    if file[-3:] == 'mp4':
        video_list.append(data_loc + file)

print(video_list)


['compressor-data/20210622-113735.88.mp4', 'compressor-data/20210613-080014.80.mp4', 'compressor-data/20210627-152829.42.mp4', 'compressor-data/20210623-095857.77.mp4', 'compressor-data/20210502-125245.03.mp4', 'compressor-data/20210627-144924.41.mp4', 'compressor-data/20210627-142618.13.mp4', 'compressor-data/20210627-114836.93.mp4', 'compressor-data/20210613-075518.27.mp4', 'compressor-data/20210623-102214.59.mp4', 'compressor-data/20210622-105209.62.mp4', 'compressor-data/20210623-072729.91.mp4', 'compressor-data/20210628-074016.54.mp4', 'compressor-data/20210627-131931.08.mp4', 'compressor-data/20210613-075851.79.mp4', 'compressor-data/20210502-124339.08.mp4', 'compressor-data/20210628-085424.39.mp4', 'compressor-data/20210627-151904.90.mp4', 'compressor-data/20210627-144952.74.mp4', 'compressor-data/20210627-140353.73.mp4', 'compressor-data/20210627-142905.77.mp4', 'compressor-data/20210502-125734.35.mp4', 'compressor-data/20210502-143623.21.mp4', 'compressor-data/20210626-151011.

In [ ]:
video_into_frames('/Users/austin/Desktop/bubble-detection/bubble-app/dist/resources/static/assets/uploads/20210502-102234.95.mp4')

In [24]:
video_into_frames(data_loc + '20210623-101817.57.mp4')

In [13]:
from IPython.display import Image

In [10]:
# contours, hier = cv2.findContours(im_gray, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
# result = cv2.imread("gray.png")

# bubbles = []
# for cnt in contours:
#     if 0<cv2.contourArea(cnt)<300:
#         cv2.drawContours(original_image, cnt, -1, (0,0,255), 2)
#         bubbles.append(cnt)


# cv2.imwrite("result.png", original_image)
# Image(filename="result.png")


In [12]:
def image_crop(image):
    black_mask = np.zeros_like(image)
    black_mask[:,:] = [255,255,255]

    # Cover top
    start_point = (0,0)
    end_point = (1280,90)
    color = (0,0,0)

    black_mask = cv2.rectangle(black_mask, start_point, end_point, color, -1)
    image = cv2.bitwise_and(image, black_mask)

    start_point = (0, 450)
    end_point = (1280, 720)
    black_mask = cv2.rectangle(black_mask, start_point, end_point, color, -1)
    image = cv2.bitwise_and(image, black_mask)

    return image

In [14]:
def grayscale_blur(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.GaussianBlur(image, (7,7), 0)
    th, image = cv2.threshold(image, 127, 255, 0)

    return image

In [46]:
def blob_extraction(image, result):
    result = cv2.connectedComponentsWithStats(result, 4, cv2.CV_32S)
    (numLabels, labels, stats, centroids) = result
    list = []
    for centroid in centroids:
        if 700 < int(centroid[0]) < 750:
            list.append(centroid[0])
    for centroid in centroids:    
        if len(list) > 10:
            cv2.circle(image, (int(centroid[0]), int(centroid[1])), 7, (0, 255, 0), 1)
    
    return image, len(centroids)

In [47]:
folder_path = "compressor-data/" + "images/"
output_path = "output_data"
image_list = sorted(os.listdir(folder_path))

for file in image_list:
    if str(file)[-3:] == "jpg":
        print(file)
        image = cv2.imread(folder_path + file)
        result = image_crop(image)
        result = grayscale_blur(result)
        image, length = blob_extraction(image, result)
        if length >= 20:
            cv2.imwrite(os.path.join(output_path, file), image)

20210623-101817.57.mp4-0-00-00.00.jpg
20210623-101817.57.mp4-0-00-00.10.jpg
20210623-101817.57.mp4-0-00-00.20.jpg
20210623-101817.57.mp4-0-00-00.40.jpg
20210623-101817.57.mp4-0-00-00.50.jpg
20210623-101817.57.mp4-0-00-00.60.jpg
20210623-101817.57.mp4-0-00-00.70.jpg
20210623-101817.57.mp4-0-00-00.80.jpg
20210623-101817.57.mp4-0-00-00.90.jpg
20210623-101817.57.mp4-0-00-01.00.jpg
20210623-101817.57.mp4-0-00-01.10.jpg
20210623-101817.57.mp4-0-00-01.20.jpg
20210623-101817.57.mp4-0-00-01.30.jpg
20210623-101817.57.mp4-0-00-01.40.jpg
20210623-101817.57.mp4-0-00-01.50.jpg
20210623-101817.57.mp4-0-00-01.60.jpg
20210623-101817.57.mp4-0-00-01.70.jpg
20210623-101817.57.mp4-0-00-01.80.jpg
20210623-101817.57.mp4-0-00-01.90.jpg
20210623-101817.57.mp4-0-00-02.00.jpg
20210623-101817.57.mp4-0-00-02.10.jpg
20210623-101817.57.mp4-0-00-02.20.jpg
20210623-101817.57.mp4-0-00-02.30.jpg
20210623-101817.57.mp4-0-00-02.40.jpg
20210623-101817.57.mp4-0-00-02.50.jpg
20210623-101817.57.mp4-0-00-02.60.jpg
20210623-101

In [3]:
image_array = []
output_path = "runs/detect/predict2/"
for file in sorted(os.listdir(output_path)):
    if str(file)[-3:] == "jpg":
        image = cv2.imread(output_path + file)
        height, width, layers = image.shape
        size = (width, height)
        image_array.append(image)

result_path = "bubble-app/dist/resources/static/assets/results/"
out = cv2.VideoWriter('./result2.mp4', cv2.VideoWriter_fourcc(*'MP4V'), 10, size)

for i in range(len(image_array)):
    out.write(image_array[i])

out.release()

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [ ]:
video_into_frames("./input.mp4")